In [1]:
from pyspark import SparkConf, SparkContext
# SparkConf : 환경 설정
# SparkContext : 실제 개발 프로그램. 드라이브
#conf = SparkConf().setMaster("yarn") # yarn으로 하면 자동으로 하둡을 사용하게 된다
conf = SparkConf().setMaster("local").setAppName("transformations_actions")
sc = SparkContext(conf=conf) # sc 변수명 바꾸면 안된다. 실무에서는 스파크 서브밋 이라는 것을 사용한다. 그때 필수

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/22 14:21:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
foods = sc.parallelize([
    "짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"
])
foods

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:287

In [3]:
foods.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [4]:
foods.countByValue()

defaultdict(int,
            {'짜장면': 4,
             '마라탕': 1,
             '짬뽕': 2,
             '떡볶이': 1,
             '쌀국수': 1,
             '라면': 2,
             '우동': 1})

In [5]:
foods.take(3)

['짜장면', '마라탕', '짬뽕']

In [6]:
foods.first()

'짜장면'

In [7]:
foods.count()

12

5. worker에서 작업하고 master에게 리턴하지 않는 함수
- foreach
- RDD에 있는 데이터를 worker에 보낼 때, 처리할 때 사용

In [8]:
foods.foreach(lambda x: print(x+" 냠냠"))

짜장면 냠냠
마라탕 냠냠
짬뽕 냠냠
떡볶이 냠냠
쌀국수 냠냠
짬뽕 냠냠
짜장면 냠냠
짜장면 냠냠
짜장면 냠냠
라면 냠냠
우동 냠냠
라면 냠냠


In [9]:
sample_rdd = sc.parallelize([1,2,3])
sample_rdd

ParallelCollectionRDD[6] at readRDDFromFile at PythonRDD.scala:287

1. map(<func>)
- 기존 RDD에서 데이터를 하나씩 꺼내서 <func>함수가 적용된 새로운 RDD를 만들어 낸다.
- func 이 곧 Task가 된다

In [10]:
sample_rdd2 = sample_rdd.map(lambda x : x+2)
sample_rdd2

PythonRDD[7] at RDD at PythonRDD.scala:53

In [11]:
sample_rdd2.collect()

[3, 4, 5]

2.'flatMap(< func >)'
- 'map'함수와 매우 흡사하나, 'flatMap'함수는 'map'의 모든 결과를 평탄화

In [12]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [13]:
moviesRDD = sc.parallelize(movies)
moviesRDD

ParallelCollectionRDD[8] at readRDDFromFile at PythonRDD.scala:287

In [14]:
'ab c'.split()

['ab', 'c']

In [15]:
mapMovies = moviesRDD.map(lambda x : x.split())
mapMovies.collect()

[['그린', '북'],
 ['매트릭스'],
 ['토이', '스토리'],
 ['캐스트', '어웨이'],
 ['포드', 'V', '페라리'],
 ['보헤미안', '랩소디'],
 ['빽', '투', '더', '퓨처'],
 ['반지의', '제왕'],
 ['죽은', '시인의', '사회']]

In [16]:
flatMovies = moviesRDD.flatMap(lambda x : x.split())
flatMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

## Wide Transformations
- 다른 파티션에 있는 (또는 다른 노드의 파티션) 데이터를 사용해서 변환하는 방법
- 데이터가 파티션을 넘나들 수 있다.
- 네트워크 통신을 동반할 가능성이 있기 때문에 느리다.

In [17]:
foods = sc.parallelize([
    "짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"
])
foods

ParallelCollectionRDD[11] at readRDDFromFile at PythonRDD.scala:287

- 'groupBy( < func > )'
    - ' < func > ' : 항상 기준을 마련해야 한다.   

In [18]:
# 첫 글자 기준
results = foods.groupBy(lambda x : x[0])

In [24]:
for (k, v) in results.collect():
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
마 ['마라탕']
짬 ['짬뽕', '짬뽕']
떡 ['떡볶이']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']


In [25]:
sc.stop()